In [ ]:
# !pip install transformers

In [3]:
import pandas as pd
import numpy as np
import pickle

from transformers import BertTokenizerFast
from torchtext import transforms as T
from konlpy.tag import Mecab

computer = 'docker'

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
if computer=='docker':
    train_data_path = '/root/share/data/train.csv'
    test_data_path = '/root/share/data/test.csv'
elif computer=='colab':
    train_data_path = ''
    test_data_path = ''

In [22]:
# data 불러오기
train = pd.read_csv(train_data_path)
# null 값 제거
train = train[train['data'].notnull()]
# //n 제거
train['data'] = train['data'].apply(lambda x: x.replace('\\n',''))

# mecab으로 tokenizing 후 token 사이에 띄어쓰기를 추가하여 문장 재구성
def tokenize(sentence, tokenizer):
    return tokenizer.morphs(sentence)
mecab = Mecab()
train['data'] = train['data'].apply(tokenize, tokenizer=mecab)
train['data'] = train['data'].apply(lambda x: ' '.join(x))

# test data도 동일하게 진행
test = pd.read_csv(test_data_path)
test['data'] = test['data'].apply(lambda x: x.replace('\\n',''))
test['data'] = test['data'].apply(tokenize, tokenizer=mecab)
test['data'] = test['data'].apply(lambda x: ' '.join(x))

In [23]:
# 학습된 bert-kor를 통해 word to token 변환
## train
tokenizer_bert = BertTokenizerFast.from_pretrained('kykim/bert-kor-base')
train_data = tokenizer_bert(train['data'].tolist(), truncation=True, padding='max_length', max_length=300)
train_data = T.ToTensor()(train_data.data['input_ids'])

## test
test_data = tokenizer_bert(test['data'].tolist(), truncation=True, padding='max_length', max_length=300)
test_data = T.ToTensor()(test_data.data['input_ids'])

In [25]:
# 데이터 저장
with open('/root/share/data/train_data.pkl','wb') as f:
    pickle.dump(train_data, f)

with open('/root/share/data/test_data.pkl','wb') as f:
    pickle.dump(test_data, f)